## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import platform
from matplotlib import rc

# 운영 체제에 따라 폰트 설정
if platform.system() == 'Windows':  # Windows 환경
    rc('font', family='Malgun Gothic')  # 맑은 고딕

# 음수 표시 가능하도록 설정
plt.rcParams['axes.unicode_minus'] = False

In [3]:
# 경고문구 숨기기
import warnings
import pandas as pd
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

반복적으로 쓰는 함수툴

In [4]:
def plot_box(df, col_name):
    """
    주어진 데이터프레임과 열 이름에 대해 박스 플롯을 그리는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    col_name (str): 열 이름
    """
    plt.figure(figsize=(7, 4))
    plt.boxplot(df[col_name], vert=False)
    plt.xlabel(col_name)
    plt.title(f'Box Plot of {col_name}')
    plt.show()

def value_counts_ratio(df, col_name, target_name='임신_성공_여부'):
    """
    주어진 데이터프레임의 특정 열에 대해 각 값마다 타겟 변수의 비율과 갯수, 총 갯수를 출력하는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    col_name (str): 열 이름
    target_name (str): 타겟 변수 이름
    """
    # 각 값마다 타겟 변수의 비율 계산
    value_counts = df.groupby(col_name)[target_name].value_counts(normalize=True).unstack().fillna(0)
    
    # 각 값마다 타겟 변수의 갯수 계산
    counts = df.groupby(col_name)[target_name].value_counts().unstack().fillna(0)
    
    # 각 값마다 총 갯수 계산
    total_counts = df[col_name].value_counts().rename('Total_Count')
    
    # 비율과 갯수를 합침
    result = value_counts.join(counts, lsuffix='_ratio', rsuffix='_count')
    
    # 총 갯수를 합침
    result = result.join(total_counts, on=col_name)
    
    # 출력 형식 조정
    result.index.name = 'variable'
    print(f"\n{col_name}별 {target_name} 비율 및 갯수\n")
    print(result.rename(columns=lambda x: x.split('_')[0]))

def summarize_group(df, group_by_columns):
    """
    주어진 데이터프레임을 특정 열들로 그룹화하여 각 그룹의 '임신_성공_여부' 비율과 갯수를 요약하는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    group_by_columns (list): 그룹화할 열 이름들의 리스트
    """
    # 데이터프레임을 그룹화
    grouped_df = df.groupby(group_by_columns)
    
    # 결과를 저장할 리스트 초기화
    results = []
    
    # 그룹화된 데이터프레임의 내용을 확인하는 코드
    for name, group in grouped_df:
        # 그룹의 갯수 계산
        group_count = group.shape[0]
        
        # '임신_성공_여부' 변수의 '임신 성공' 비율과 갯수 계산
        success_count = group['임신_성공_여부'].value_counts().get(1, 0)
        success_ratio = success_count / group_count
        
        # 결과 리스트에 추가
        results.append([name, success_count, success_ratio, group_count])
    
    # 결과 리스트를 데이터프레임으로 변환
    results_df = pd.DataFrame(results, columns=['group', "'임신_성공_여부' count", 'ratio', 'Total'])
    
    # 그룹화된 변수들의 이름을 제목행으로 출력
    print(f"Grouped by: {', '.join(group_by_columns)}")
    print()
    # 데이터프레임 출력
    print(results_df)

def plot_ratio(df, group_by_column, target_column='임신_성공_여부', success_value=1):
    """
    주어진 데이터프레임을 특정 열로 그룹화하여 각 그룹의 타겟 변수의 성공 비율을 시각화하는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    group_by_column (str): 그룹화할 열 이름
    target_column (str): 타겟 변수 이름
    success_value (int): 타겟 변수의 성공을 나타내는 값
    """
    # 데이터프레임을 그룹화
    grouped_df = df.groupby(group_by_column)
    
    # 결과를 저장할 리스트 초기화
    results = []
    
    # 그룹화된 데이터프레임의 내용을 확인하는 코드
    for name, group in grouped_df:
        # 그룹의 갯수 계산
        group_count = group.shape[0]
        
        # '임신_성공_여부' 변수의 '성공' 비율과 갯수 계산
        success_count = group[target_column].value_counts().get(success_value, 0)
        success_ratio = success_count / group_count
        
        # 결과 리스트에 추가
        results.append([name, success_count, success_ratio, group_count])
    
    # 결과 리스트를 데이터프레임으로 변환
    results_df = pd.DataFrame(results, columns=['group', f"'{success_value}' count", 'ratio', 'Total'])
    
    # 그래프 크기 설정
    plt.figure(figsize=(10, 5))
    
    # 막대 그래프 생성
    ax = results_df.plot(kind='bar', x='group', y='ratio', legend=False)
    
    # 각 막대 위에 임신 성공 여부 갯수와 총 갯수 표시
    for i, (success_count, total) in enumerate(zip(results_df[f"'{success_value}' count"], results_df['Total'])):
        ax.text(i, results_df['ratio'][i], f'{success_count} ({total})', ha='center', va='bottom', fontsize=8)
    
    # 그래프 제목 및 축 레이블 설정
    ax.set_title(f'{success_value} Ratio by {group_by_column}')
    ax.set_xlabel(group_by_column)
    ax.set_ylabel(f'{success_value} Ratio')
   
    # 그래프 출력
    plt.show()

def plot_ratio_boxplot(data, column_name, target_column='임신_성공_여부'):
    """
    주어진 데이터프레임의 특정 열과 타겟 변수 간의 관계를 박스 플롯으로 시각화하는 함수.

    Parameters:
    data (pd.DataFrame): 데이터프레임
    column_name (str): 열 이름
    target_column (str): 타겟 변수 이름
    """

    # 그래프 그리기
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=column_name, y=target_column, data=data)

    # 그래프 제목 및 레이블 설정
    plt.title(f'{column_name} vs {target_column}')
    plt.xlabel(column_name)
    plt.ylabel(target_column)

    # 그래프 출력
    plt.show()

### Data Load

In [5]:
# 데이터 로드
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

# 변수명에서 공백을 밑줄로 대체
train.columns = train.columns.str.replace(' ', '_')
test.columns = test.columns.str.replace(' ', '_')

In [6]:
# train.info(verbose=True)

## EDA

In [7]:
# '시술_유형' 변수의 값이 'DI'에 해당 안하는 데이터 추출
DI_train = train[train['시술_유형'] == 'DI'] 
DI_test = test[test['시술_유형'] == 'DI']

# 모든 값이 결측값인 변수를 제거
DI_train = DI_train.dropna(axis=1, how='all')
DI_test = DI_test.dropna(axis=1, how='all')

# 결측값이 없는 상태에서 고유값이 1개만 존재하는 변수 제거
unique_value_columns_train = DI_train.columns[DI_train.nunique() <= 1]
unique_value_columns_test = DI_test.columns[DI_test.nunique() <= 1]

train = DI_train.drop(columns=unique_value_columns_train)
test = DI_test.drop(columns=unique_value_columns_test)

타겟변수 비율 확인

In [8]:
# '임신_성공_여부' 변수의 값 개수 계산
value_counts = train['임신_성공_여부'].value_counts()

# 각 값이 전체에서 차지하는 비율 계산
value_percentages = value_counts / len(train) * 100

# 결과 출력
print("임신_성공_여부 값 개수:")
print(value_counts)
print("\n임신_성공_여부 값 비율 (%):")
print(value_percentages)

임신_성공_여부 값 개수:
임신_성공_여부
0    5480
1     811
Name: count, dtype: int64

임신_성공_여부 값 비율 (%):
임신_성공_여부
0    87.108568
1    12.891432
Name: count, dtype: float64


EDA를 통해 무조건 불임인 데이터를 제거하여 EDA 재진행

In [9]:
# '임신_시도_또는_마지막_임신_경과_연수' 값이 20 이상인 행 제거 (결측값은 유지)
train_sub_1 = train[train['임신_시도_또는_마지막_임신_경과_연수'] >= 20]
train = train[(train['임신_시도_또는_마지막_임신_경과_연수'] < 20) | (train['임신_시도_또는_마지막_임신_경과_연수'].isna())]

# 부부_주_불임_원인과 부부_부_불임_원인 값이 모두 1인 행 제거
train_sub_2 = train[(train['부부_주_불임_원인'] == 1) & (train['부부_부_불임_원인'] == 1)]
train = train[~((train['부부_주_불임_원인'] == 1) & (train['부부_부_불임_원인'] == 1))]

# 특정_시술_유형 값이 'GIFT'인 행 제거
train_sub_3 = train[train['특정_시술_유형'] == 'GIFT']
train = train[train['특정_시술_유형'] != 'GIFT']

# 'IVF_임신_횟수' 변수의 값이 '3회'인 행 제거
train_sub_4 = train[train['IVF_임신_횟수'] == '3회']
train = train[train['IVF_임신_횟수'] != '3회']

# 제거된 행을 train_sub에 저장
train_sub = pd.concat([train_sub_1, train_sub_2, train_sub_3, train_sub_4])

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6283 entries, 14 to 256326
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     6283 non-null   object 
 1   시술_시기_코드               6283 non-null   object 
 2   시술_당시_나이               6283 non-null   object 
 3   임신_시도_또는_마지막_임신_경과_연수  299 non-null    float64
 4   특정_시술_유형               6283 non-null   object 
 5   배란_자극_여부               6283 non-null   int64  
 6   남성_주_불임_원인             6283 non-null   int64  
 7   남성_부_불임_원인             6283 non-null   int64  
 8   여성_주_불임_원인             6283 non-null   int64  
 9   여성_부_불임_원인             6283 non-null   int64  
 10  부부_주_불임_원인             6283 non-null   int64  
 11  부부_부_불임_원인             6283 non-null   int64  
 12  불명확_불임_원인              6283 non-null   int64  
 13  불임_원인_-_난관_질환          6283 non-null   int64  
 14  불임_원인_-_남성_요인          6283 non-null   int64  
 15  불임_원인_

### 1. 시술 시기 코드

난임 시술을 받은 시기를 기준으로 코드 부여

In [11]:
value_counts_ratio(train, '시술_시기_코드')


시술_시기_코드별 임신_성공_여부 비율 및 갯수

                 0         1    0    1  Total
variable                                     
TRCMWS    0.862651  0.137349  716  114    830
TRDQAZ    0.887967  0.112033  856  108    964
TRJXFG    0.886635  0.113365  743   95    838
TRVNRY    0.877301  0.122699  858  120    978
TRXQMD    0.837626  0.162374  748  145    893
TRYBLT    0.865340  0.134660  739  115    854
TRZKPL    0.876890  0.123110  812  114    926


In [12]:
# '시술_시기_코드' 변수 제거
train = train.drop(columns=['시술_시기_코드'])
test = test.drop(columns=['시술_시기_코드'])

### 2. 시술 당시 나이

환자의 시술 당시 나이(연령대)

In [13]:
value_counts_ratio(train, '시술_당시_나이')


시술_당시_나이별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
만18-34세   0.805408  0.194592  1668  403   2071
만35-37세   0.848757  0.151243  1229  219   1448
만38-39세   0.897338  0.102662   944  108   1052
만40-42세   0.930165  0.069835   959   72   1031
만43-44세   0.980907  0.019093   411    8    419
만45-50세   0.996183  0.003817   261    1    262


In [14]:
# '시술_당시_나이' 변수의 값을 중앙값으로 변환하는 함수 정의
def convert_age_category(age_category):
    if age_category == '만18-34세':
        return (18 + 34) / 2
    elif age_category == '만35-37세':
        return (35 + 37) / 2
    elif age_category == '만38-39세':
        return (38 + 39) / 2
    elif age_category == '만40-42세':
        return (40 + 42) / 2
    elif age_category == '만43-44세':
        return (43 + 44) / 2
    elif age_category == '만45-50세':
        return (45 + 50) / 2
    else:
        return None

# '시술_당시_나이' 변수의 값을 중앙값으로 변환
train['시술_당시_나이'] = train['시술_당시_나이'].apply(convert_age_category)
test['시술_당시_나이'] = test['시술_당시_나이'].apply(convert_age_category)

In [15]:
value_counts_ratio(train, '시술_당시_나이')


시술_당시_나이별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
26.0      0.805408  0.194592  1668  403   2071
36.0      0.848757  0.151243  1229  219   1448
38.5      0.897338  0.102662   944  108   1052
41.0      0.930165  0.069835   959   72   1031
43.5      0.980907  0.019093   411    8    419
47.5      0.996183  0.003817   261    1    262


### 3. 임신 시도 또는 마지막 임신 경과 일수

환자가 처음 임신을 시도한 시점 또는 마지막 임신 이후 현재까지의 경과 연수 (년 단위)

In [16]:
value_counts_ratio(train, '임신_시도_또는_마지막_임신_경과_연수')


임신_시도_또는_마지막_임신_경과_연수별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
1.0       1.000000  0.000000   2.0  0.0      2
3.0       0.666667  0.333333   6.0  3.0      9
4.0       0.700000  0.300000   7.0  3.0     10
5.0       0.785714  0.214286  11.0  3.0     14
6.0       0.852941  0.147059  29.0  5.0     34
7.0       0.727273  0.272727  24.0  9.0     33
8.0       0.853659  0.146341  35.0  6.0     41
9.0       0.965517  0.034483  28.0  1.0     29
10.0      0.774194  0.225806  24.0  7.0     31
11.0      0.862069  0.137931  25.0  4.0     29
12.0      0.863636  0.136364  19.0  3.0     22
13.0      0.888889  0.111111  16.0  2.0     18
14.0      0.800000  0.200000   8.0  2.0     10
15.0      0.800000  0.200000   4.0  1.0      5
16.0      1.000000  0.000000   1.0  0.0      1
17.0      0.833333  0.166667   5.0  1.0      6
18.0      1.000000  0.000000   2.0  0.0      2
19.0      0.666667  0.333333   2.0  1.0      3


In [17]:
print("결측값 개수:", train['임신_시도_또는_마지막_임신_경과_연수'].isnull().sum())

train['임신_시도_또는_마지막_임신_경과_연수'].fillna(-1, inplace=True)
test['임신_시도_또는_마지막_임신_경과_연수'].fillna(-1, inplace=True)

print("결측값 개수:", train['임신_시도_또는_마지막_임신_경과_연수'].isnull().sum())

결측값 개수: 5984
결측값 개수: 0


In [18]:
value_counts_ratio(train, '임신_시도_또는_마지막_임신_경과_연수')


임신_시도_또는_마지막_임신_경과_연수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1  Total
variable                                          
-1.0      0.872995  0.127005  5224.0  760.0   5984
 1.0      1.000000  0.000000     2.0    0.0      2
 3.0      0.666667  0.333333     6.0    3.0      9
 4.0      0.700000  0.300000     7.0    3.0     10
 5.0      0.785714  0.214286    11.0    3.0     14
 6.0      0.852941  0.147059    29.0    5.0     34
 7.0      0.727273  0.272727    24.0    9.0     33
 8.0      0.853659  0.146341    35.0    6.0     41
 9.0      0.965517  0.034483    28.0    1.0     29
 10.0     0.774194  0.225806    24.0    7.0     31
 11.0     0.862069  0.137931    25.0    4.0     29
 12.0     0.863636  0.136364    19.0    3.0     22
 13.0     0.888889  0.111111    16.0    2.0     18
 14.0     0.800000  0.200000     8.0    2.0     10
 15.0     0.800000  0.200000     4.0    1.0      5
 16.0     1.000000  0.000000     1.0    0.0      1
 17.0     0.833333  0.166667     5.0    

### 5. 특정_시술_유형

GIFT – 생식세포 나팔관 이식  
Generic DI – 기증 정자 주입  
ICI – 자궁경부 인공 수정  
IUI – 인공 수정 (자궁 내 주입)  
IVI – 질 내 인공 수정   
Unknown – 시술 유형 정보가 제공되지 않았거나 알 수 없음  

In [19]:
summarize_group(train, ['특정_시술_유형'])

Grouped by: 특정_시술_유형

           group  '임신_성공_여부' count     ratio  Total
0  (Generic DI,)                12  0.169014     71
1         (ICI,)                13  0.135417     96
2         (IUI,)               784  0.128672   6093
3         (IVI,)                 2  0.086957     23


In [20]:
test['특정_시술_유형'].value_counts()

특정_시술_유형
IUI           2112
ICI             28
Generic DI      26
IVI              8
IUI:ICI          2
Name: count, dtype: int64

test 데이터에 없고 train 데이터에만 있는 행 제거

In [21]:
train['특정_시술_유형'].isnull().sum()

0

In [22]:
# 시술 유형 변환 함수
def transform_treatment(treatment):
    if isinstance(treatment, str):  # treatment가 문자열인지 확인
        if treatment == "IUI:ICI":
            return "IUI"
    return treatment  # 기타 시술은 그대로 반환

# 변환된 열 추가
train['특정_시술_유형'] = train['특정_시술_유형'].apply(transform_treatment)
test['특정_시술_유형'] = test['특정_시술_유형'].apply(transform_treatment)

In [23]:
value_counts_ratio(train, '특정_시술_유형')


특정_시술_유형별 임신_성공_여부 비율 및 갯수

                   0         1     0    1  Total
variable                                        
Generic DI  0.830986  0.169014    59   12     71
ICI         0.864583  0.135417    83   13     96
IUI         0.871328  0.128672  5309  784   6093
IVI         0.913043  0.086957    21    2     23


In [24]:
test['특정_시술_유형'].value_counts()

특정_시술_유형
IUI           2114
ICI             28
Generic DI      26
IVI              8
Name: count, dtype: int64

### 6. 배란_자극_여부

배란 자극 치료가 사용되었는지 여부

In [25]:
value_counts_ratio(train, '배란_자극_여부')


배란_자극_여부별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
0         0.882412  0.117588  2649  353   3002
1         0.860408  0.139592  2823  458   3281


### 11. 남성_주_불임_원인

남성의 주요 불임 원인 여부

In [26]:
value_counts_ratio(train, '남성_주_불임_원인')


남성_주_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
0         0.874167  0.125833  5245  755   6000
1         0.802120  0.197880   227   56    283


### 12. 남성_부_불임_원인

남성의 부차적 불임 원인 여부

In [27]:
value_counts_ratio(train, '남성_부_불임_원인')


남성_부_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
0         0.870921  0.129079  5364  795   6159
1         0.870968  0.129032   108   16    124


In [28]:
summarize_group(train, ['남성_주_불임_원인', '남성_부_불임_원인'])

Grouped by: 남성_주_불임_원인, 남성_부_불임_원인

    group  '임신_성공_여부' count     ratio  Total
0  (0, 0)               739  0.125766   5876
1  (0, 1)                16  0.129032    124
2  (1, 0)                56  0.197880    283


### 13. 여성_주_불임_원인

여성의 주요 불임 원인 여부

In [29]:
value_counts_ratio(train, '여성_주_불임_원인')


여성_주_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
0         0.873831  0.126169  5139  742   5881
1         0.828358  0.171642   333   69    402


### 14. 여성_부_불임_원인

여성의 부차적 불임 원인 여부

In [30]:
value_counts_ratio(train, '여성_부_불임_원인')


여성_부_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
0         0.871303  0.128697  5362  792   6154
1         0.852713  0.147287   110   19    129


In [31]:
summarize_group(train, ['여성_주_불임_원인', '여성_부_불임_원인'])

Grouped by: 여성_주_불임_원인, 여성_부_불임_원인

    group  '임신_성공_여부' count     ratio  Total
0  (0, 0)               723  0.125695   5752
1  (0, 1)                19  0.147287    129
2  (1, 0)                69  0.171642    402


### 15. 부부_주_불임_원인

부부의 주요 불임 원인 여부

In [32]:
value_counts_ratio(train, '부부_주_불임_원인')


부부_주_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
0         0.874810  0.125190  5178  741   5919
1         0.807692  0.192308   294   70    364


### 16. 부부_부_불임_원인

부부의 부차적 불임 원인 여부

In [33]:
value_counts_ratio(train, '부부_부_불임_원인')


부부_부_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
0         0.871020  0.128980  5416  802   6218
1         0.861538  0.138462    56    9     65


In [34]:
summarize_group(train, ['부부_주_불임_원인','부부_부_불임_원인'])

Grouped by: 부부_주_불임_원인, 부부_부_불임_원인

    group  '임신_성공_여부' count     ratio  Total
0  (0, 0)               732  0.125043   5854
1  (0, 1)                 9  0.138462     65
2  (1, 0)                70  0.192308    364


### 17. 불명확_불임_원인

불명의 원인으로 인한 불임 여부

In [35]:
value_counts_ratio(train, '불명확_불임_원인')


불명확_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
0         0.868068  0.131932  5040  766   5806
1         0.905660  0.094340   432   45    477


### 18. 불임_원인_-_난관_질환

난관 이상으로 인한 불임 여부

In [36]:
value_counts_ratio(train, '불임_원인_-_난관_질환')


불임_원인_-_난관_질환별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
0         0.870287  0.129713  5401  805   6206
1         0.922078  0.077922    71    6     77


### 19. 불임_원인_-_남성_요인

남성 요인으로 인한 불임 여부

In [37]:
value_counts_ratio(train, '불임_원인_-_남성_요인')


불임_원인_-_남성_요인별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
0         0.895925  0.104075  3056  355   3411
1         0.841226  0.158774  2416  456   2872


### 20. 불임_원인_-_배란_장애

배란 장애로 인한 불임 여부

In [38]:
value_counts_ratio(train, '불임_원인_-_배란_장애')


불임_원인_-_배란_장애별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
0         0.871039  0.128961  5140  761   5901
1         0.869110  0.130890   332   50    382


### 23. 불임_원인_-_자궁내막증

자궁내막증으로 인한 불임 여부

In [39]:
value_counts_ratio(train, '불임_원인_-_자궁내막증')


불임_원인_-_자궁내막증별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
0         0.869608  0.130392  5322  798   6120
1         0.920245  0.079755   150   13    163


### 24. 불임_원인_-_정자_농도

파트너의 정자 농도 문제 여부

In [40]:
value_counts_ratio(train, '불임_원인_-_정자_농도')


불임_원인_-_정자_농도별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
0         0.871397  0.128603  5441  803   6244
1         0.794872  0.205128    31    8     39


In [41]:
test['불임_원인_-_정자_농도'].value_counts()

불임_원인_-_정자_농도
0    2165
1      11
Name: count, dtype: int64

### 29. 총_시술_횟수

IVF 및 DI 포함한 전체 시술 횟수

In [42]:
value_counts_ratio(train, '총_시술_횟수')


총_시술_횟수별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
0회        0.926768  0.073232   367   29    396
1회        0.865406  0.134594   778  121    899
2회        0.866184  0.133816   958  148   1106
3회        0.876923  0.123077   912  128   1040
4회        0.853524  0.146476   775  133    908
5회        0.870095  0.129905   643   96    739
6회 이상     0.869456  0.130544  1039  156   1195


### 30. 클리닉_내_총_시술_횟수

특정 클리닉에서 받은 IVF 및 DI 시술 횟수

In [43]:
value_counts_ratio(train, '클리닉_내_총_시술_횟수')


클리닉_내_총_시술_횟수별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
0회        0.894336  0.105664   821   97    918
1회        0.870229  0.129771  1026  153   1179
2회        0.872414  0.127586  1012  148   1160
3회        0.875135  0.124865   813  116    929
4회        0.867591  0.132409   688  105    793
5회        0.864865  0.135135   544   85    629
6회 이상     0.841481  0.158519   568  107    675


### 31. IVF_시술_횟수

이전에 수행된 IVF 시술 횟수

In [44]:
value_counts_ratio(train, 'IVF_시술_횟수')


IVF_시술_횟수별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
0회        0.861037  0.138963  3222  520   3742
1회        0.871795  0.128205   816  120    936
2회        0.898990  0.101010   534   60    594
3회        0.878282  0.121718   368   51    419
4회        0.891473  0.108527   230   28    258
5회        0.920732  0.079268   151   13    164
6회 이상     0.888235  0.111765   151   19    170


### 32. DI_시술_횟수

이전에 수행된 DI 시술 횟수

In [45]:
value_counts_ratio(train, 'DI_시술_횟수')


DI_시술_횟수별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
0회        0.900885  0.099115  1018  112   1130
1회        0.873262  0.126738  1068  155   1223
2회        0.860670  0.139330   976  158   1134
3회        0.871397  0.128603   786  116    902
4회        0.857343  0.142657   613  102    715
5회        0.890052  0.109948   510   63    573
6회 이상     0.826733  0.173267   501  105    606


### 33. 총_임신_횟수

IVF 및 DI 포함한 총 임신 횟수

In [46]:
value_counts_ratio(train, '총_임신_횟수')


총_임신_횟수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1  Total
variable                                          
0회        0.902459  0.097541  3377.0  365.0   3742
1회        0.830102  0.169898  1798.0  368.0   2166
2회        0.785276  0.214724   256.0   70.0    326
3회        0.840909  0.159091    37.0    7.0     44
4회        1.000000  0.000000     4.0    0.0      4
5회        0.000000  1.000000     0.0    1.0      1


### 34. IVF_임신_횟수

IVF 시술로 인한 임신 횟수

In [47]:
value_counts_ratio(train, 'IVF_임신_횟수')


IVF_임신_횟수별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
0회        0.869432  0.130568  5094  765   5859
1회        0.888041  0.111959   349   44    393
2회        0.935484  0.064516    29    2     31


### 35. DI_임신_횟수

DI 시술로 인한 임신 횟수

In [48]:
value_counts_ratio(train, 'DI_임신_횟수')


DI_임신_횟수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1  Total
variable                                          
0회        0.900656  0.099344  3708.0  409.0   4117
1회        0.823656  0.176344  1532.0  328.0   1860
2회        0.751880  0.248120   200.0   66.0    266
3회        0.810811  0.189189    30.0    7.0     37
4회        1.000000  0.000000     2.0    0.0      2
5회        0.000000  1.000000     0.0    1.0      1


### 36. 총_출산_횟수

IVF 및 DI 시술로 출산한 횟수

In [49]:
value_counts_ratio(train, '총_출산_횟수')


총_출산_횟수별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
0회        0.899144  0.100856  3887  436   4323
1회        0.809045  0.190955  1449  342   1791
2회        0.800000  0.200000   120   30    150
3회        0.842105  0.157895    16    3     19


### 37. IVF_출산_횟수

IVF 시술로 출산한 횟수

In [50]:
value_counts_ratio(train, 'IVF_출산_횟수')


IVF_출산_횟수별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
0회        0.870259  0.129741  5232  780   6012
1회        0.889734  0.110266   234   29    263
2회        0.750000  0.250000     6    2      8


### 38. DI_출산_횟수

DI 시술로 출산한 횟수

In [51]:
value_counts_ratio(train, 'DI_출산_횟수')


DI_출산_횟수별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
0회        0.898209  0.101791  4112  466   4578
1회        0.797558  0.202442  1241  315   1556
2회        0.798507  0.201493   107   27    134
3회        0.800000  0.200000    12    3     15


시술 횟수 -> 숫자 매핑

In [52]:
# 숫자 매핑
mapping = {
    '0회': 0, '1회': 1, '2회': 2, '3회': 3, 
    '4회': 4, '5회': 5, '6회 이상': 6
}

# 변환할 열 목록
columns_to_map = [
    '총_시술_횟수', '클리닉_내_총_시술_횟수', 'IVF_시술_횟수', 
    'DI_시술_횟수', '총_임신_횟수', 'IVF_임신_횟수', 'DI_임신_횟수', 
    '총_출산_횟수', 'IVF_출산_횟수', 'DI_출산_횟수'
]

# 숫자로 변환
for col in columns_to_map:
    train[col] = train[col].map(mapping)
    test[col] = test[col].map(mapping)

In [53]:
# 비율 계산 (train 데이터프레임)
train['IVF_임신_시술_비율'] = np.where(train['IVF_시술_횟수'] > 0, train['IVF_임신_횟수'] / train['IVF_시술_횟수'], 0)
train['DI_임신_시술_비율'] = np.where(train['DI_시술_횟수'] > 0, train['DI_임신_횟수'] / train['DI_시술_횟수'], 0)
train['임신_성공률'] = np.where(train['총_시술_횟수'] > 0, train['총_임신_횟수'] / train['총_시술_횟수'], 0)

# 비율 계산 (test 데이터프레임)
test['IVF_임신_시술_비율'] = np.where(test['IVF_시술_횟수'] > 0, test['IVF_임신_횟수'] / test['IVF_시술_횟수'], 0)
test['DI_임신_시술_비율'] = np.where(test['DI_시술_횟수'] > 0, test['DI_임신_횟수'] / test['DI_시술_횟수'], 0)
test['임신_성공률'] = np.where(test['총_시술_횟수'] > 0, test['총_임신_횟수'] / test['총_시술_횟수'], 0)

In [54]:
# 변수 리스트
columns = [
    # '클리닉_내_총_시술_횟수', 
    '총_시술_횟수', '총_출산_횟수', '총_임신_횟수', '임신_성공률',
    'IVF_시술_횟수', 'IVF_임신_횟수', 'IVF_출산_횟수', 'IVF_임신_시술_비율',
    # 'DI_시술_횟수', 'DI_임신_횟수', 'DI_출산_횟수', 'DI_임신_시술_비율',
]

# 변수 제거
train = train.drop(columns=columns)
test = test.drop(columns=columns)

### 56. 정자_기증자_나이

정자 기증자의 등록 당시 나이

In [55]:
value_counts_ratio(train, '정자_기증자_나이')


정자_기증자_나이별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
만20세 이하   0.859935  0.140065   264   43    307
만21-25세   0.867670  0.132330  1095  167   1262
만26-30세   0.873191  0.126809  1026  149   1175
만31-35세   0.867320  0.132680   974  149   1123
만36-40세   0.883860  0.116140  1035  136   1171
만41-45세   0.863938  0.136062   781  123    904
알 수 없음    0.870968  0.129032   297   44    341


### 68. 임신_성공_여부

해당 시술로부터 임신 성공 여부

In [56]:
train['임신_성공_여부'].value_counts()

임신_성공_여부
0    5472
1     811
Name: count, dtype: int64

기타 처리

In [57]:
# DI_train과 DI_test의 열 비교
train_columns = set(train.columns)
test_columns = set(test.columns)

# DI_train에는 있지만 DI_test에는 없는 열 찾기
missing_in_test = train_columns - test_columns

print("DI_train에는 있지만 DI_test에는 없는 열:")
print(missing_in_test)

DI_train에는 있지만 DI_test에는 없는 열:
{'임신_성공_여부', '불임_원인_-_정자_형태'}


In [58]:
# '불임_원인_-_정자_형태' 변수 제거
train.drop('불임_원인_-_정자_형태', axis=1, inplace=True)

## 변수 상태 확인

In [59]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6283 entries, 14 to 256326
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     6283 non-null   object 
 1   시술_당시_나이               6283 non-null   float64
 2   임신_시도_또는_마지막_임신_경과_연수  6283 non-null   float64
 3   특정_시술_유형               6283 non-null   object 
 4   배란_자극_여부               6283 non-null   int64  
 5   남성_주_불임_원인             6283 non-null   int64  
 6   남성_부_불임_원인             6283 non-null   int64  
 7   여성_주_불임_원인             6283 non-null   int64  
 8   여성_부_불임_원인             6283 non-null   int64  
 9   부부_주_불임_원인             6283 non-null   int64  
 10  부부_부_불임_원인             6283 non-null   int64  
 11  불명확_불임_원인              6283 non-null   int64  
 12  불임_원인_-_난관_질환          6283 non-null   int64  
 13  불임_원인_-_남성_요인          6283 non-null   int64  
 14  불임_원인_-_배란_장애          6283 non-null   int64  
 15  불임_원인_

In [60]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2176 entries, 26 to 90046
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     2176 non-null   object 
 1   시술_당시_나이               2176 non-null   float64
 2   임신_시도_또는_마지막_임신_경과_연수  2176 non-null   float64
 3   특정_시술_유형               2176 non-null   object 
 4   배란_자극_여부               2176 non-null   int64  
 5   남성_주_불임_원인             2176 non-null   int64  
 6   남성_부_불임_원인             2176 non-null   int64  
 7   여성_주_불임_원인             2176 non-null   int64  
 8   여성_부_불임_원인             2176 non-null   int64  
 9   부부_주_불임_원인             2176 non-null   int64  
 10  부부_부_불임_원인             2176 non-null   int64  
 11  불명확_불임_원인              2176 non-null   int64  
 12  불임_원인_-_난관_질환          2176 non-null   int64  
 13  불임_원인_-_남성_요인          2176 non-null   int64  
 14  불임_원인_-_배란_장애          2176 non-null   int64  
 15  불임_원인_-

In [61]:
train_sub.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, 5112 to 176952
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     8 non-null      object 
 1   시술_시기_코드               8 non-null      object 
 2   시술_당시_나이               8 non-null      object 
 3   임신_시도_또는_마지막_임신_경과_연수  4 non-null      float64
 4   특정_시술_유형               8 non-null      object 
 5   배란_자극_여부               8 non-null      int64  
 6   남성_주_불임_원인             8 non-null      int64  
 7   남성_부_불임_원인             8 non-null      int64  
 8   여성_주_불임_원인             8 non-null      int64  
 9   여성_부_불임_원인             8 non-null      int64  
 10  부부_주_불임_원인             8 non-null      int64  
 11  부부_부_불임_원인             8 non-null      int64  
 12  불명확_불임_원인              8 non-null      int64  
 13  불임_원인_-_난관_질환          8 non-null      int64  
 14  불임_원인_-_남성_요인          8 non-null      int64  
 15  불임_원인_-

## 데이터 저장

In [63]:
len(train_sub)

8

In [62]:
# '임신_성공_여부' 변수의 값 개수 계산
value_counts = train_sub['임신_성공_여부'].value_counts()

# 각 값이 전체에서 차지하는 비율 계산
value_percentages = value_counts / len(train_sub) * 100

# 결과 출력
print("임신_성공_여부 값 개수:")
print(value_counts)
print("\n임신_성공_여부 값 비율 (%):")
print(value_percentages)

임신_성공_여부 값 개수:
임신_성공_여부
0    8
Name: count, dtype: int64

임신_성공_여부 값 비율 (%):
임신_성공_여부
0    100.0
Name: count, dtype: float64


In [186]:
# test 데이터셋을 CSV 파일로 저장 (UTF-8 인코딩)
train.to_csv('../data/DI_train_dataset_50.csv', index=False, encoding='utf-8-sig')
test.to_csv('../data/DI_test_dataset_50.csv', index=False, encoding='utf-8-sig')
train_sub.to_csv('../data/DI_train_sub_dataset_50.csv', index=False, encoding='utf-8-sig')

.